### Parkrun Distances

This notebook calculates the road distance between two specified parkruns using the Google Distance Martrix API. Mean emmissions 221.4g/mile.

In [13]:
# import packages and read data

import pandas as pd
import json
import requests
from geopy import distance
from geopy.geocoders import GoogleV3
import math

df = pd.read_csv('data/uk-parkruns.csv')
df.columns = ['name', 'location', 'lon', 'lat']
df.head()


,name,location,lon,lat
0,Aberdare,Aberdare Park,-3.460089,51.718796
1,Aberdeen,Aberdeen Beach Esplanade,-2.086256,57.173898
2,Aberystwyth juniors,Plas Crug Park,-4.078402,52.412885
3,Aberystwyth,Plascrug Park,-4.080401,52.414546
4,Abingdon,"Rye Meadow, Abingdon",-1.277429,51.667957


In [3]:
def get_key(path, index):
    """Gets api key from a json file.

    Args:
        path (string): the file path to the json file containing the api keys
        index (int): the index of the key to be returned

    Returns:
        string: the key at the index specified
    """
    with open(path) as f:
        return list(json.load(f).values())[index]

KEY_PATH = "credentials.json"
key = get_key(KEY_PATH, 0)


In [19]:
def get_parkrun(name):
    """Gets the coordinates of a specified parkrun

    Args:
        name (string): the case-sensitive name of the parkrun

    Returns:
        lat, lon: the latitude and longitude of the parkrun
    """
    try:
        lat = df[df.name == name].lat.item()
        lon = df[df.name == name].lon.item()
        print(f"Coordinates of {name} parkrun: {lat}, {lon}")
        return lat, lon
    except:
        print("Parkrun not found")


def get_postcode(postcode, key):
    """Gets the coordinates of a specified postcode

    Args:
        postcode (string): the postcode
        key (string): API key for access to Google Maps Geocoder API

    Returns:
        lat, lon: the latitude and longitude of the postcode
    """
    try:
        g = GoogleV3(key)
        location = g.geocode(postcode)
        lat = location.latitude
        lon = location.longitude
        print(f"Coordinates of {postcode}: {lat}, {lon}")
        return lat, lon
    except:
        print('Postcode coordinates not found')


def get_road_distance(lat1, lon1, lat2, lon2, key):
    """Gets the distance between two locations using the Google Maps Distance Matrix API

    Args:
        lat1: latitude of the first location
        lon1 (_type_): longitude of the first location
        lat2 (_type_): latitude of the second location
        lon2 (_type_): longitude of the second location
        key (_type_): API key for access to Google Maps Distance Matrix API

    Returns:
        data: the distance between the two locations
    """
    try:
        url = f"https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial&origins={lat1},{lon1}&destinations={lat2}%2C{lon2}&key={key}"
        r = requests.get(url)
        data = r.json()
        return data
    except:
        print('Route not found')


def get_geo_dist(lat1, lon1, lat2, lon2):
    """Gets the geodesic distance between two locations

    Args:
        lat1: the latitude of the first location
        lon1: the longitude of the first location
        lat2: the latitude of the second location
        lon2: the longitude of the second location
    
    Returns:
        the geodesic distance between the two locations in miles to two decimal places
    """
    return round(distance.distance((lat1, lon1), (lat2, lon2)).miles, 2)



In [12]:
LOC_1 = "EH13 0HT"
LOC_2 = "Edinburgh"

lat1, lon1 = get_postcode(LOC_1, key)
lat2, lon2 = get_parkrun(LOC_2)

distance = get_road_distance(lat1, lon1, lat2, lon2, key)
if distance is not None:
    print(
        f"Road distance between {LOC_1} and {LOC_2} parkrun: {distance['rows'][0]['elements'][0]['distance']['text'].replace(' mi', '')} miles")

    

Coordinates of EH13 0HT: 55.9038479, -3.270493
Coordinates of Edinburgh parkrun: 55.978806, -3.291585
Road distance between EH13 0HT and Edinburgh parkrun: 8.5 miles


In [31]:
# closest parkrun that begins with given letter

letter = "A"
starts_with = df[df['name'].str.startswith(letter)]
print(starts_with)

starts_with['dist'] = get_geo_dist(starts_with['lat'], starts_with['lon'], lat1, lon1)

get_geo_dist(lat1, lon1, lat2, lon2)

starts_with.head()

                            name                                  location  \
0                       Aberdare                             Aberdare Park   
1                       Aberdeen                  Aberdeen Beach Esplanade   
2            Aberystwyth juniors                            Plas Crug Park   
3                    Aberystwyth                             Plascrug Park   
4                       Abingdon                      Rye Meadow, Abingdon   
5                  Acton juniors                                Acton Park   
6                          Agnew                                Agnew Park   
7                         Albert                Albert Park, Middlesbrough   
8                       Aldenham                     Aldenham Country Park   
9                 Alderford Lake                            Alderford Lake   
10     Alderman Kneeshaw juniors       Alderman Kneeshaw Recreation Ground   
11  Alexandra juniors, Moss Side                            Alex

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().